In [2]:

import rebel_decode as rd
import pandas as pd
import networkx as nx


file_number = str(3).zfill(4) # 0001-0010
public_file = os.path.join(directory, f"{file_number}_public.txt")
truth_file = os.path.join(directory, f"{file_number}_truth.txt")

p_info = rd.parse_public_data(public_file) # "0001_public.txt"
REBS=p_info.get_rebs()
COT=p_info.get_cot() # df 
NEA=p_info.get_nea() # df
LOC=p_info.get_loc() # df
FLAVOUR_DICT=p_info.get_flavour_dict()

truth = rd.parse_truth_data(truth_file) # "../out/0001_truth.txt"
star_coords = truth.get_stars()
ship_movements = truth.get_moves()
messages = truth.get_messages()

rebs_df = pd.read_csv(public_file,
                header=None, engine='python',
                sep='t=(\d+), (\w+), (\w+), (.*)').dropna(how='all', axis=1) # regex from rebel_decode.py
rebs_df.columns=['t', 'msg_type', 'messenger', 'msg_content']
rebs_df.reset_index(drop=True)

rebs_df = rebs_df[['t','messenger','msg_type']]
rebs_df = rebs_df.set_index('t')\
            .groupby('messenger')\
            .apply(lambda df_x: df_x.reindex(range(1, 1000+1)))\
            .drop('messenger', axis=1).reset_index()

relations = nx.from_pandas_edgelist(COT, source='messenger', target='cotraveller')
ships = {rebel: ship for ship, shipnumber in enumerate((nx.connected_components(relations))) for rebel in shipnumber}

rebs_df['ship'] = rebs_df['messenger'].map(ships)


ModuleNotFoundError: No module named 'rebel_decode'